In [2]:
!pip install pyspark

    100% |████████████████████████████████| 211.9MB 4.0kB/s eta 0:00:01   19% |██████▍                         | 41.9MB 45.5MB/s eta 0:00:04�██████████▍                 | 95.6MB 49.9MB/s eta 0:00:03    55% |█████████████████▊              | 117.5MB 61.0MB/s eta 0:00:02    | 120.4MB 59.7MB/s eta 0:00:02    58% |██████████████████▋             | 123.2MB 40.9MB/s eta 0:00:03    | 125.1MB 56.3MB/s eta 0:00:02�████████▎      | 167.6MB 55.6MB/s eta 0:00:017MB 55.5MB/s eta 0:00:01�██████▊      | 170.1MB 58.3MB/s eta 0:00:01��████████████████████████▉     | 177.8MB 59.2MB/s eta 0:00:01�█████████████▎    | 180.6MB 27.7MB/s eta 0:00:020MB 61.6MB/s eta 0:00:01███████████████████████████▊    | 183.9MB 61.1MB/s eta 0:00:01███████████████████████████▍  | 194.3MB 31.3MB/s eta 0:00:01.4MB 60.7MB/s eta 0:00:01��████████████  | 198.4MB 59.9MB/s eta 0:00:01�� | 200.3MB 60.1MB/s eta 0:00:01█████████████▍ | 201.1MB 58.4MB/s eta 0:00:01�████▌ | 202.1MB 30.7MB/s eta 0:00:01�█████████████████████████▉ | 204.3

In [3]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model

In [4]:
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_4d0a1baf7c2d4528be3a7696716f4836 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-cNeD8bYaZl_hW1OofQRjanOhDUXZ-jViUTATOFWu0Yq',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_4d0a1baf7c2d4528be3a7696716f4836.get_object(Bucket='titanicc901411227974723b98e915e35dbb583',Key='titanic.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [6]:
df_data_1.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
df_data_1 = df_data_1.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark_df = spark.createDataFrame(df_data_1)

splitted_data = spark_df.randomSplit([0.85, 0.15], 48)
train_data = splitted_data[0]
test_data = splitted_data[1]

In [9]:
sex_indexer = StringIndexer(inputCol="Sex", outputCol="sex_code")
features_assembler = VectorAssembler(inputCols=["sex_code", "Pclass", "Age", "SibSp", "Parch", "Fare"], outputCol="features")

estimator = RandomForestClassifier(labelCol="Survived", featuresCol="features")

In [12]:
pipeline = Pipeline(stages=[sex_indexer, features_assembler, estimator])

In [13]:
model = pipeline.fit(train_data)

In [14]:
predictions = model.transform(test_data)
evaluator = BinaryClassificationEvaluator(labelCol="Survived", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.841477
Test Error = 0.158523


In [15]:
new_data = [{'Pclass':1, 'Sex':'female', 'Age':26, 'SibSp':0, 'Parch':0, 'Fare':20},
            {'Pclass':3,'Sex': 'male', 'Age':26, 'SibSp':0, 'Parch':0, 'Fare':20}]

new_df = spark.createDataFrame(new_data)
new_df.show()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pyspark/sql/session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---+----+-----+------+------+-----+
|Age|Fare|Parch|Pclass|   Sex|SibSp|
+---+----+-----+------+------+-----+
| 26|  20|    0|     1|female|    0|
| 26|  20|    0|     3|  male|    0|
+---+----+-----+------+------+-----+



In [16]:
new_predictions = model.transform(new_df)
new_predictions.select("prediction").show()

+----------+
|prediction|
+----------+
|       1.0|
|       0.0|
+----------+

